This block of code is responsible for building the index for our text corpus. We use BERT for our embeddings model and tokenizer, and we use FAISS cosine similarity to index our normalized vectors.

In [1]:
import os
# import sys

# from src.processing import mean_pooling, mean_pooling_embedding_with_normalization

# sys.path.append('.')

import argparse
import json
import numpy as np

import faiss
import torch
# from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

#either musique dataset or 2wikimultihopqa dataset
# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--dataset',type = str)

# args = parser.parse_args()

dim = 768
#normalize embeddings before building index using inner product. Note that maximal inner product with normalized embeddings is equivalent to cosine similarity 
norm = True


"""CHANGE THIS LINE TO USE OTHER DATASET"""
dataset = 'musique'
"""CHANGE THIS LINE TO USE OTHER DATASET"""


model_label = 'facebook_contriever'
vector_path = f'data/{dataset}/{dataset}_{model_label}_proposition_vectors_norm.npy'
index_path = f'data/{dataset}/{dataset}_{model_label}_proposition_ip_norm.index'
if(os.path.isfile(vector_path)):
    vectors = np.load(vector_path)
if dataset == 'musique':
    corpus = json.load(open('data/musique_proposition_corpus.json', 'r'))
elif dataset == '2wikimultihopqa':
    corpus = json.load(open('data/2wikimultihopqa_proposition_corpus.json', 'r'))
corpus_contents = []
for item in corpus:
    corpus_contents.append(item['title'] + '\n' + item['propositions'])
print('corpus size: {}'.format(len(corpus_contents)))

#create sentence-level embeddings using mean-pooling and normalize to prepare for cosine similarity indexing
#note: UPDATE TO USE distributedDataParallel

def mean_pooling(tokenEmbeddings, paddingInfo):
    tokenEmbeddingsNoPad = tokenEmbeddings.masked_fill(~paddingInfo[...,None].bool(), 0)
    sentenceEmbeddings = tokenEmbeddingsNoPad.sum(dim = 1) / paddingInfo.sum(dim = 1)[...,None]
    return sentenceEmbeddings

def mean_pooling_embedding_with_normalization(batch_str, tokenizer, model):
    mps_device = torch.device("mps") 
    encoding = tokenizer(batch_str, padding=True, truncation=True, return_tensors='pt').to(mps_device)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    input_ids = input_ids.to(mps_device)
    attention_mask = attention_mask.to(mps_device)
    outputs = model(input_ids, attention_mask=attention_mask)
    sentenceEmbeddings = mean_pooling(outputs[0], attention_mask)
    sentenceEmbeddingsNorm = sentenceEmbeddings.divide(torch.linalg.norm(sentenceEmbeddings,dim = 1)[...,None])
    return sentenceEmbeddingsNorm

if os.path.isfile(vector_path):
    print('Loading existing vectors:', vector_path)
    vectors = np.load(vector_path)
    print('Vectors loaded:', len(vectors))

else:
    # load model
    tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
    model = AutoModel.from_pretrained('facebook/contriever')
    # Check if multiple GPUs are available and if so, use them all
    if (torch.cuda.is_available()):
        device = torch.device("cuda")    
        model.to(device)
        model = torch.nn.DataParallel(model)
    #test batch size = 16 and batch size = 32 
    batch_size = 16
    vectors = np.zeros((len(corpus_contents), dim))
    #get batch_size number of entries from corpus_contents, tokenize and embed them in 768 dimensional space
    for idx in range(0, len(corpus_contents), batch_size):
        end_idx = min(idx + batch_size, len(corpus_contents))
        seqs = corpus_contents[idx:end_idx]
        try:
            #read above comments to understand what this function does
            batch_embeddings = mean_pooling_embedding_with_normalization(seqs, tokenizer, model)
        except Exception as e:
            batch_embeddings = torch.zeros((len(seqs), dim))
            print(f'Error at {idx}:', e)
        vectors[idx:end_idx] = batch_embeddings.detach().to('cpu').numpy()
    print("Type of vectors is {}".format(type(vectors)))
    fp = open(vector_path, 'wb')
    np.save(fp, vectors)
    fp.close()
    print('vectors saved to {}'.format(vector_path))

    #using FAISS on CPU (GPU support unavailable for mac)
    if os.path.isfile(index_path):
            print('index file already exists:', index_path)
            print('index size: {}'.format(faiss.read_index(index_path).ntotal))
    else:
        print('Building index...')
        index = faiss.IndexFlatIP(dim)
        vectors = vectors.astype('float32')
        index.add(vectors)

        # save faiss index to file
        # fp = open(index_path, 'w')
        faiss.write_index(index, index_path)
        print('index saved to {}'.format(index_path))
        print('index size: {}'.format(index.ntotal))

/Users/amarkanaka/miniconda3/envs/faiss_1.8.0_new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


corpus size: 11656
Loading existing vectors: data/musique/musique_facebook_contriever_proposition_vectors_norm.npy
Vectors loaded: 11656


Sanity Check that Indexing Worked

In [3]:
D, I = index.search(vectors[:5], 4) # sanity check
print(I)
print(D)

NameError: name 'index' is not defined

Run the following block if you want to know statistics about approximate number of tokens in each line of corpus

In [8]:
# total_len = 0
# max_len = 0
# min_len = 1000000
# for line in corpus_contents:
#     total_len += len(line)
#     if len(line) > max_len:
#         max_len = len(line)
#     if len(line) < min_len:
#         min_len = len(line)
# print(max_len / 4)
# print(min_len / 4)
# print((total_len / len(corpus_contents)) / 4)    

525.25
31.25
149.56335792724778


The following blocks of code are responsible for evaluating our RAG system's retrieval on our two corpora

In [3]:
from abc import abstractmethod
class DocumentRetriever:
    @abstractmethod
    def rank_docs(self, query: str, top_k: int):
        """
        Rank the documents in the corpus based on the given query
        :param query:
        :param top_k: 
        :return: ranks and scores of the retrieved documents
        """
class DPRRetriever(DocumentRetriever):
    def __init__(self, model_name: str, faiss_index: str, corpus):
        """

        :param model_name:
        :param faiss_index: The path to the faiss index
        """
        device  = torch.device("mps") 
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.faiss_index = faiss_index
        self.corpus = corpus
        self.device = device

    def rank_docs(self, query: str, top_k: int):
        # query_embedding = mean_pooling_embedding(query, self.tokenizer, self.model, self.device)
        with torch.no_grad():
            query_embedding = mean_pooling_embedding_with_normalization(query, self.tokenizer, self.model).detach().cpu().numpy()
        inner_product, corpus_idx = self.faiss_index.search(query_embedding, top_k)
        return corpus_idx.tolist()[0], inner_product.tolist()[0]

In [301]:
import time
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
import tiktoken
import openai

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

def num_tokens_by_tiktoken(text: str):
    return len(enc.encode(text))

def merge_elements_with_same_first_line(elements, prefix='Wikipedia Title: '):
    merged_dict = {}

    # Iterate through each element in the list
    for element in elements:
        # Split the element into lines and get the first line
        lines = element.split('\n')
        first_line = lines[0]

        # Check if the first line is already a key in the dictionary
        if first_line in merged_dict:
            # Append the current element to the existing value
            merged_dict[first_line] += "\n" + element.split(first_line, 1)[1].strip('\n')
        else:
            # Add the current element as a new entry in the dictionary
            merged_dict[first_line] = prefix + element

    # Extract the merged elements from the dictionary
    merged_elements = list(merged_dict.values())
    return merged_elements

ircot_reason_instruction = 'You serve as an intelligent assistant, adept at facilitating users through complex, multi-hop reasoning across multiple documents. Your task is to generate the next thought for the current step, DON\'T generate all thoughts at once! If you reach what you believe to be the final step, start with "So the answer is:".'

def reason_step(dataset, few_shot: list, query: str, passages: list, thoughts: list, client):
    """
    Given few-shot samples, query, previous retrieved passages, and previous thoughts, generate the next thought with LangChain LLM.
    The generated thought is used for further retrieval step.
    :return: next thought
    """
    prompt_demo = ''

    prompt_user = ''
    if dataset in ['hotpotqa']:
        passages = merge_elements_with_same_first_line(passages)
        
    #put into the format of documents in IRCOT tests
    for passage in passages:
        prompt_user += f'Wikipedia Title: {passage}\n\n'
    prompt_user += f'Question: {query}\nThought:' + ' '.join(thoughts)

    # print("Passages: ")
    # print(passages)
    
    # print("Prompt User:")
    # print(prompt_user)

    # print("Cur_sample:")

    examples = []
    for sample in few_shot:
        cur_sample = f'{sample["document"]}\n\nQuestion: {sample["question"]}\nThought: {sample["thought_and_answer"]}\n\n'
        if num_tokens_by_tiktoken(ircot_reason_instruction + prompt_demo + cur_sample + prompt_user) < 15000:
                examples.append({"input": f'{sample["document"]}\n\nQuestion: {sample["question"]}\nThought: '+sample["thought"], "output": sample["thought_and_answer"]})

    # print(examples)

    # print(cur_sample)

    # print("Prompt Demo:")
    # print(prompt_demo)

    example_prompt = ChatPromptTemplate.from_messages(
        [
            ("human", "{input}"),
            ("ai", "{output}"),
        ]
    )

    # print("input")
    # print(f'{sample["document"]}\n\nQuestion: {sample["question"]}')

    # print("output")
    # print(sample["thought_and_answer"])



    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=examples,
    )

    final_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", ircot_reason_instruction),
            few_shot_prompt,
            ("human", "{input}"),
        ]
    )


#     messages = ChatPromptTemplate([("system", ircot_reason_instruction + '\n\n' + prompt_demo),
#                                                  ("human", "{prompt_user}")])
    # print(messages.to_messages())

    for retry in range(5):
        try:
            chain = final_prompt | client
            chat_completion = chain.invoke({"input":prompt_user})
            response_content = chat_completion.content
            return response_content
            # print(response_content)
        except openai.RateLimitError as e:
            print("Rate Limit Exceeded. Trying again")
            wait_time = 2 ** retry  # Exponential backoff
            time.sleep(wait_time)

In [263]:
def retrieve_step(query: str, corpus, top_k: int, retriever: DocumentRetriever, dataset: str):
    doc_ids, scores = retriever.rank_docs(query, top_k=top_k)
    if dataset in ['hotpotqa']:
        retrieved_passages = []
        for doc_id in doc_ids:
            key = list(corpus.keys())[doc_id]
            retrieved_passages.append(key + '\n' + ''.join(corpus[key]))
    elif dataset in ['musique', '2wikimultihopqa']:        
        retrieved_passages = [corpus[doc_id]['title'] + '\n' + corpus[doc_id]['text'] for doc_id in doc_ids]
    else:
        raise NotImplementedError(f'Dataset {dataset} not implemented')
    return retrieved_passages, scores

def process_sample(idx, sample, dataset, top_k, k_list,max_steps, few_shot_samples, corpus, retriever, client, processed_ids):
    # Check if the sample has already been processed
    if dataset in ['hotpotqa', '2wikimultihopqa']:
        sample_id = sample['_id']
    elif dataset in ['musique']:
        sample_id = sample['id']
    else:
        raise NotImplementedError(f'Dataset {dataset} not implemented')
    if sample_id in processed_ids:
        return  # Skip already processed samples
    else:
        processed_ids.add(sample_id)

    # Perform retrieval and reasoning steps
    query = sample['question']
    #uncomment this line if you want to see the questions being asked
    # print(query)
    retrieved_passages, scores = retrieve_step(query, corpus, top_k, retriever, dataset)

    thoughts = []
    retrieved_passages_dict = {passage: score for passage, score in zip(retrieved_passages, scores)}
    it = 1
    for it in range(1, max_steps):
        print("in IRCOT loop")
        # print("retrieved passages:")
        # print(retrieved_passages[:top_k])
        # reason_step(dataset, few_shot_samples, query, retrieved_passages[:top_k], thoughts, client)
        new_thought = reason_step(dataset, few_shot_samples, query, retrieved_passages[:top_k], thoughts, client)
        thoughts.append(new_thought)
        if 'So the answer is:' in new_thought:
            break
        new_retrieved_passages, new_scores = retrieve_step(new_thought, corpus, top_k, retriever, dataset)

        for passage, score in zip(new_retrieved_passages, new_scores):
            if passage in retrieved_passages_dict:
                retrieved_passages_dict[passage] = max(retrieved_passages_dict[passage], score)
            else:
                retrieved_passages_dict[passage] = score

        retrieved_passages, scores = zip(*retrieved_passages_dict.items())

        sorted_passages_scores = sorted(zip(retrieved_passages, scores), key=lambda x: x[1], reverse=True)
        retrieved_passages, scores = zip(*sorted_passages_scores)
    # end iteration

    # calculate recall
    if dataset in ['hotpotqa']:
        gold_passages = [item for item in sample['supporting_facts']]
        gold_items = set([item[0] for item in gold_passages])
        retrieved_items = [passage.split('\n')[0].strip() for passage in retrieved_passages]
    elif dataset in ['musique']:
        gold_passages = [item for item in sample['paragraphs'] if item['is_supporting']]
        print(gold_passages)
        gold_items = set([item['title'] + '\n' + item['paragraph_text'] for item in gold_passages])
        # print(gold_items)
        retrieved_items = retrieved_passages
    elif dataset in ['2wikimultihopqa']:
        gold_passages = [item for item in sample['supporting_facts']]
        gold_items = set([item[0] for item in gold_passages])
        retrieved_items = [passage.split('\n')[0].strip() for passage in retrieved_passages]
    else:
        raise NotImplementedError(f'Dataset {dataset} not implemented')

    recall = dict()
    print(f'idx: {idx + 1} ', end='')
    for k in k_list:
        # in the top k retrieved docs, sum the number of true positives (gold items) found and divide by the total number of true positives
        # fraction of retrieved passages found
        recall[k] = sum(1 for t in gold_items if t in retrieved_items[:k]) / len(gold_items)
    return idx, recall, retrieved_passages, thoughts, it

In [6]:
def parse_prompt(file_path: str, has_context=True):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Split the content by the metadata pattern
    parts = content.split('# METADATA: ')
    parsed_data = []
    if has_context:
        for part in parts[1:]:  # Skip the first split as it will be empty
            metadata_section, rest_of_data = part.split('\n', 1)
            metadata = json.loads(metadata_section)
            document_sections = rest_of_data.strip().split('\n\nQ: ')
            document_text = document_sections[0].strip()
            qa_pair = document_sections[1].split('\nA: ')
            question = qa_pair[0].strip()
            thought_and_answer = qa_pair[1].strip().split('So the answer is: ')
            thought = thought_and_answer[0].strip()
            answer = thought_and_answer[1].strip()

            parsed_data.append({
                'metadata': metadata,
                'document': document_text,
                'question': question,
                'thought_and_answer': qa_pair[1].strip(),
                'thought': thought,
                'answer': answer
            })
    else:
        for part in parts[1:]:
            metadata_section, rest_of_data = part.split('\n', 1)
            metadata = json.loads(metadata_section)
            s = rest_of_data.split('\n')
            question = s[0][3:].strip()
            thought_and_answer = s[1][3:].strip().split('So the answer is: ')
            thought = thought_and_answer[0].strip()
            answer = thought_and_answer[1].strip()

            parsed_data.append({
                'metadata': metadata,
                'question': question,
                'thought_and_answer': s[1][3:].strip(),
                'thought': thought,
                'answer': answer
            })

    return parsed_data


In [115]:
llm_model = 'gpt-3.5-turbo-1106'
llm = 'openai'
"""
User-inputted args
"""

"""
For 2wikimultihopqa, change max_steps to 2, num_demo to 1 to perform multistep retrieval.
For musique, change max_steps to 4, num_demo 1 to perform multistep retrieval.
"""
max_steps = 4
num_demo = 1
top_k = 8
#load dataset
if dataset == 'musique':
    data = json.load(open('data/musique.json', 'r'))
    corpus = json.load(open('data/musique_corpus.json', 'r'))
    prompt_path = 'data/ircot_prompts/musique/gold_with_3_distractors_context_cot_qa_codex.txt'
    max_steps = max_steps if max_steps is not None else 4
elif dataset == '2wikimultihopqa':
    data = json.load(open('data/2wikimultihopqa.json', 'r'))
    corpus = json.load(open('data/2wikimultihopqa_corpus.json', 'r'))
    prompt_path = 'data/ircot_prompts/2wikimultihopqa/gold_with_3_distractors_context_cot_qa_codex.txt'
    max_steps = max_steps if max_steps is not None else 2
else:
    raise NotImplementedError(f'Dataset {dataset} not implemented')


In [299]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
#Create OpenAI Client

client = ChatOpenAI(api_key=os.environ.get("OPENAI_API_KEY"), model=llm_model, temperature=0.0, max_retries=5, timeout=60)

In [12]:
#load num_demo distractor passages
few_shot_samples = parse_prompt(prompt_path)
few_shot_samples = few_shot_samples[:num_demo]
print('num of demo:', len(few_shot_samples))

num of demo: 1


In [13]:
# doc_ensemble_str = 'doc_ensemble' if doc_ensemble else 'no_ensemble'
doc_ensemble_str = ''
alt_model_label = 'facebook_contriever'
if max_steps > 1:
    output_path = f'output/ircot/{dataset}_{alt_model_label}_demo_{num_demo}_{llm_model}_{doc_ensemble_str}_step_{max_steps}_top_{top_k}.json'
else:  # only one step
    top_k = 100
    output_path = f'output/proposition_{dataset}_{alt_model_label}_{doc_ensemble_str}.json'

In [14]:
if dataset == 'musique':
    faiss_index = faiss.read_index('data/musique/musique_facebook_contriever_proposition_ip_norm.index')
else:
    faiss_index = faiss.read_index('data/2wikimultihopqa/2wikimultihopqa_facebook_contriever_proposition_ip_norm.index')

In [15]:
model_label = 'facebook/contriever'
retriever = DPRRetriever(model_label, faiss_index, corpus)

Evaluate retrieval

In [16]:
k_list = [1, 2, 5, 10, 15, 20, 30, 50, 100]
total_recall = {k: 0 for k in k_list}

In [63]:
results = data

In [302]:
processed_ids = set()

In [303]:
k_list = [1, 2, 5, 8]
total_recall = {k: 0 for k in k_list}
processed_ids = set()
for idx, sample in enumerate(data):
    idx, recall, retrieved_passages, thoughts, it = process_sample(idx, sample, dataset, top_k, k_list,max_steps, few_shot_samples, corpus, retriever, client, processed_ids) 
    
    # print metrics
    for k in k_list:
        total_recall[k] += recall[k]
        print(f'R@{k}: {total_recall[k] / (idx + 1):.4f} ', end='')
    print()
    if max_steps > 1:
        print('[ITERATION]', it, '[PASSAGE]', len(retrieved_passages), '[THOUGHT]', thoughts)
    
    # record results
    results[idx]['retrieved'] = retrieved_passages
    results[idx]['recall'] = recall
    results[idx]['thoughts'] = thoughts
        
    # if idx % 50 == 0:
    #     f = open(output_path, 'w')
    #     json.dump(results, f)
    #     f.close()

# save results
f = open(output_path, 'w')
json.dump(results, f)
f.close()
print(f'Saved results to {output_path}')
for k in k_list:
    #average recall (across 1,000 questions for musique)
    print(f'R@{k}: {total_recall[k] / len(data):.4f} ', end='')

in IRCOT loop
in IRCOT loop
in IRCOT loop
[{'idx': 1, 'title': 'FC Barcelona', 'paragraph_text': "Despite being the favourites and starting strongly, Barcelona finished the 2006–07 season without trophies. A pre-season US tour was later blamed for a string of injuries to key players, including leading scorer Eto'o and rising star Lionel Messi. There was open feuding as Eto'o publicly criticized coach Frank Rijkaard and Ronaldinho. Ronaldinho also admitted that a lack of fitness affected his form. In La Liga, Barcelona were in first place for much of the season, but inconsistency in the New Year saw Real Madrid overtake them to become champions. Barcelona advanced to the semi-finals of the Copa del Rey, winning the first leg against Getafe 5–2, with a goal from Messi bringing comparison to Diego Maradona's goal of the century, but then lost the second leg 4–0. They took part in the 2006 FIFA Club World Cup, but were beaten by a late goal in the final against Brazilian side Internacional

In [239]:
few_shot_samples[0]

{'metadata': {'qid': '2hop__292995_8796'},
 'document': 'Wikipedia Title: The Last Horse\nThe Last Horse (Spanish:El último caballo) is a 1950 Spanish comedy film directed by Edgar Neville starring Fernando Fernán Gómez.\n\nWikipedia Title: Southampton\nThe University of Southampton, which was founded in 1862 and received its Royal Charter as a university in 1952, has over 22,000 students. The university is ranked in the top 100 research universities in the world in the Academic Ranking of World Universities 2010. In 2010, the THES - QS World University Rankings positioned the University of Southampton in the top 80 universities in the world. The university considers itself one of the top 5 research universities in the UK. The university has a global reputation for research into engineering sciences, oceanography, chemistry, cancer sciences, sound and vibration research, computer science and electronics, optoelectronics and textile conservation at the Textile Conservation Centre (which